In [1]:
def check_seasonality(y, path_x13as, dep_var_transform='dlog'):
    import os
    from statsmodels import tsa as tsa
    X13PATH= os.chdir(path_x13as)
    if dep_var_transform =='diff':
        x13results=tsa.x13.x13_arima_analysis(endog=y, x12path=X13PATH,  forecast_periods=60, prefer_x13=True, retspec=True, log=False)
    else:
        x13results=tsa.x13.x13_arima_analysis(endog=y, x12path=X13PATH,  forecast_periods=60, prefer_x13=True, retspec=True, log=True)

    # Checking presence of seasonality
    list_results = x13results.results.splitlines()
    row_num = list_results.index('  COMBINED TEST FOR THE PRESENCE OF IDENTIFIABLE SEASONALITY')
    seasonality_result = list_results[row_num + 3].strip()
    print(seasonality_result)
    return seasonality_result

In [ ]:
# Function returns d_10 - seasonal component actuals and forecasts and d_11 deseasonalised series actuals
def calculate_seasonality(y, path_x13as, dep_var_transform='dlog'):
    # X13PATH is the directory of your local folder x13as
    import os
    import pandas as pd
    from statsmodels import tsa as tsa
    X13PATH= os.chdir(path_x13as)

    # getting the seasonally adjusted y (y_d11)
    if dep_var_transform =='diff':
        x13results=tsa.x13.x13_arima_analysis(endog=y, x12path=X13PATH,  forecast_periods=60, prefer_x13=True, retspec=True, log=False)
    else:
        x13results=tsa.x13.x13_arima_analysis(endog=y, x12path=X13PATH,  forecast_periods=60, prefer_x13=True, retspec=True, log=True)
    y_d11=pd.Series(x13results.seasadj.values, index=y.index.values)
    
    # Extracting seasonal components forecast
    list_results = x13results.results.splitlines()
    start_row = list_results.index(' D 10.A  Final seasonal component forecasts')
    
    # Extracting seasonal components forecast
    seasonal_factors = {}
    date_mapping = {1:'01', 2:'02', 3:'03', 4:'04',5:'05',6:'06',7:'07',8:'08',9:'09',10:'10',11:'11',12:'12'}
    for i in range(7,20,3):
        row_num = start_row + i
        row_num1 = start_row+1 + i
        row_raw=list_results[row_num]
        row_raw1=list_results[row_num1]
        row1 = list_results[row_num].split()[:-1]
        row2 = list_results[row_num1].split()
        row_fin=row1+row2
        for i in range(1,len(row_fin)):
            akey = row_fin[0] + date_mapping[i + (13-len(row_fin))]
            seasonal_factors[akey] = float(row_fin[i])

    seas_factors = pd.Series(seasonal_factors)
    seas_factors.index = pd.to_datetime(seas_factors.index , format = '%Y%m')
    
    # Extracting seasonal components actuals
    start_row_act = list_results.index(' D 10  Final seasonal factors')
    seasonal_factors_act = {}
    date_mapping = {1:'01', 2:'02', 3:'03', 4:'04',5:'05',6:'06',7:'07',8:'08',9:'09',10:'10',11:'11',12:'12'}
    for i in range(8,28,3):
        row_num_act = start_row_act + i
        row_num1_act = start_row_act+1 + i
        row_raw_act=list_results[row_num_act]
        row_raw1_act=list_results[row_num1_act]
        row1_act = list_results[row_num_act].split()[:-1]
        row2_act = list_results[row_num1_act].split()
        row_fin_act=row1_act+row2_act
        for i in range(1,len(row_fin_act)):
            akey = row_fin_act[0] + date_mapping[i + (13-len(row_fin_act))]
            seasonal_factors_act[akey] = float(row_fin_act[i])

    seas_factors_act = pd.Series(seasonal_factors_act)
    seas_factors_act.index = pd.to_datetime(seas_factors_act.index , format = '%Y%m')
    return seas_factors, seas_factors_act, y_d11 